In [25]:
# Patched version of TwitterIE to extract metadata of tweets with no videos 
# Removed the `No video could be found in this tweet` error
import re

from yt_dlp.extractor.twitter import TwitterIE, TwitterCardIE, PeriscopeIE, TwitterBroadcastIE, TwitterSpacesIE
from yt_dlp import YoutubeDL
from yt_dlp.utils import ( 
    ExtractorError,
    float_or_none,
    format_field,
    int_or_none,
    make_archive_id,
    str_or_none,
    traverse_obj,
    try_get,
    unified_timestamp,
    update_url_query,
    url_or_none,
)

class TwitterIEPatched(TwitterIE):
    def _real_extract(self, url):
        twid, selected_index = self._match_valid_url(url).group('id', 'index')
        status = self._extract_status(twid)

        title = description = traverse_obj(
            status, (('full_text', 'text'), {lambda x: x.replace('\n', ' ')}), get_all=False) or ''
        title = re.sub(r'\s+(https?://[^ ]+)', '', title)
        user = status.get('user') or {}
        uploader = user.get('name')
        if uploader:
            title = f'{uploader} - {title}'
        uploader_id = user.get('screen_name')

        info = {
            'id': twid,
            'title': title,
            'description': description,
            'uploader': uploader,
            'timestamp': unified_timestamp(status.get('created_at')),
            'uploader_id': uploader_id,
            'uploader_url': format_field(uploader_id, None, 'https://twitter.com/%s'),
            'like_count': int_or_none(status.get('favorite_count')),
            'repost_count': int_or_none(status.get('retweet_count')),
            'comment_count': int_or_none(status.get('reply_count')),
            'age_limit': 18 if status.get('possibly_sensitive') else 0,
            'tags': traverse_obj(status, ('entities', 'hashtags', ..., 'text')),
        }

        def extract_from_video_info(media):
            media_id = traverse_obj(media, 'id_str', 'id', expected_type=str_or_none)
            self.write_debug(f'Extracting from video info: {media_id}')

            formats = []
            subtitles = {}
            for variant in traverse_obj(media, ('video_info', 'variants', ...)):
                fmts, subs = self._extract_variant_formats(variant, twid)
                subtitles = self._merge_subtitles(subtitles, subs)
                formats.extend(fmts)

            thumbnails = []
            media_url = media.get('media_url_https') or media.get('media_url')
            if media_url:
                def add_thumbnail(name, size):
                    thumbnails.append({
                        'id': name,
                        'url': update_url_query(media_url, {'name': name}),
                        'width': int_or_none(size.get('w') or size.get('width')),
                        'height': int_or_none(size.get('h') or size.get('height')),
                    })
                for name, size in media.get('sizes', {}).items():
                    add_thumbnail(name, size)
                add_thumbnail('orig', media.get('original_info') or {})

            return {
                'id': media_id,
                'formats': formats,
                'subtitles': subtitles,
                'thumbnails': thumbnails,
                'view_count': traverse_obj(media, ('mediaStats', 'viewCount', {int_or_none})),
                'duration': float_or_none(traverse_obj(media, ('video_info', 'duration_millis')), 1000),
                # The codec of http formats are unknown
                '_format_sort_fields': ('res', 'br', 'size', 'proto'),
            }

        def extract_from_card_info(card):
            if not card:
                return

            self.write_debug(f'Extracting from card info: {card.get("url")}')
            binding_values = card['binding_values']

            def get_binding_value(k):
                o = binding_values.get(k) or {}
                return try_get(o, lambda x: x[x['type'].lower() + '_value'])

            card_name = card['name'].split(':')[-1]
            if card_name == 'player':
                yield {
                    '_type': 'url',
                    'url': get_binding_value('player_url'),
                }
            elif card_name == 'periscope_broadcast':
                yield {
                    '_type': 'url',
                    'url': get_binding_value('url') or get_binding_value('player_url'),
                    'ie_key': PeriscopeIE.ie_key(),
                }
            elif card_name == 'broadcast':
                yield {
                    '_type': 'url',
                    'url': get_binding_value('broadcast_url'),
                    'ie_key': TwitterBroadcastIE.ie_key(),
                }
            elif card_name == 'audiospace':
                yield {
                    '_type': 'url',
                    'url': f'https://twitter.com/i/spaces/{get_binding_value("id")}',
                    'ie_key': TwitterSpacesIE.ie_key(),
                }
            elif card_name == 'summary':
                yield {
                    '_type': 'url',
                    'url': get_binding_value('card_url'),
                }
            elif card_name == 'unified_card':
                unified_card = self._parse_json(get_binding_value('unified_card'), twid)
                yield from map(extract_from_video_info, traverse_obj(
                    unified_card, ('media_entities', ...), expected_type=dict))
            # amplify, promo_video_website, promo_video_convo, appplayer,
            # video_direct_message, poll2choice_video, poll3choice_video,
            # poll4choice_video, ...
            else:
                is_amplify = card_name == 'amplify'
                vmap_url = get_binding_value('amplify_url_vmap') if is_amplify else get_binding_value('player_stream_url')
                content_id = get_binding_value('%s_content_id' % (card_name if is_amplify else 'player'))
                formats, subtitles = self._extract_formats_from_vmap_url(vmap_url, content_id or twid)

                thumbnails = []
                for suffix in ('_small', '', '_large', '_x_large', '_original'):
                    image = get_binding_value('player_image' + suffix) or {}
                    image_url = image.get('url')
                    if not image_url or '/player-placeholder' in image_url:
                        continue
                    thumbnails.append({
                        'id': suffix[1:] if suffix else 'medium',
                        'url': image_url,
                        'width': int_or_none(image.get('width')),
                        'height': int_or_none(image.get('height')),
                    })

                yield {
                    'formats': formats,
                    'subtitles': subtitles,
                    'thumbnails': thumbnails,
                    'duration': int_or_none(get_binding_value(
                        'content_duration_seconds')),
                }

        videos = traverse_obj(status, (
            (None, 'quoted_status'), 'extended_entities', 'media', lambda _, m: m['type'] != 'photo', {dict}))

        if self._yes_playlist(twid, selected_index, video_label='URL-specified video number'):
            selected_entries = (*map(extract_from_video_info, videos), *extract_from_card_info(status.get('card')))
        else:
            desired_obj = traverse_obj(status, (
                (None, 'quoted_status'), 'extended_entities', 'media', int(selected_index) - 1, {dict}), get_all=False)
            if not desired_obj:
                raise ExtractorError(f'Video #{selected_index} is unavailable', expected=True)
            elif desired_obj.get('type') != 'video':
                raise ExtractorError(f'Media #{selected_index} is not a video', expected=True)
            # Restore original archive id and video index in title
            for index, entry in enumerate(videos, 1):
                if entry.get('id') != desired_obj.get('id'):
                    continue
                if index == 1:
                    info['_old_archive_ids'] = [make_archive_id(self, twid)]
                if len(videos) != 1:
                    info['title'] += f' #{index}'
                break

            return {**info, **extract_from_video_info(desired_obj), 'display_id': twid}

        entries = [{**info, **data, 'display_id': twid} for data in selected_entries]
        if not entries:
            expanded_url = traverse_obj(status, ('entities', 'urls', 0, 'expanded_url'), expected_type=url_or_none)
            return self.url_result(expanded_url, display_id=twid, **info)

        entries[0]['_old_archive_ids'] = [make_archive_id(self, twid)]

        if len(entries) == 1:
            return entries[0]

        for index, entry in enumerate(entries, 1):
            entry['title'] += f' #{index}'

        return self.playlist_result(entries, **info)

In [27]:
from rich import print
downloader = YoutubeDL()
tie = TwitterIEPatched(downloader)
res = tie.extract("https://twitter.com/jedisct1/status/1772647350554464448")
print(res)

[twitter] Extracting URL: https://twitter.com/jedisct1/status/1772647350554464448
[twitter] 1772647350554464448: Downloading guest token
[twitter] 1772647350554464448: Downloading GraphQL JSON


{
    'display_id': '1772647350554464448',
    'id': '1772647350554464448',
    'title': 'Frank ⚡ - First practical SHA-256 collision for 31 steps. #fse2024',
    'description': 'First practical SHA-256 collision for 31 steps. #fse2024 https://t.co/qBo3tOLPGB',
    'uploader': 'Frank ⚡',
    'timestamp': 1711467037.0,
    'uploader_id': 'jedisct1',
    'uploader_url': 'https://twitter.com/jedisct1',
    'like_count': 2762,
    'repost_count': 521,
    'comment_count': 76,
    'age_limit': 0,
    'tags': ['fse2024'],
    '_type': 'url',
    'url': None
}

In [37]:
import dotenv
from twikit import Client
import os

dotenv.load_dotenv()
client = Client()
client.login(auth_info_1=os.getenv("TWITTER_USERNAME"), auth_info_2=os.getenv("TWITTER_EMAIL"), password=os.getenv("TWITTER_PASSWORD"))
res = client.get_tweet_by_id("1773207638698078280")
print(res)

<Tweet id="1773207638698078280">

In [42]:
res = client.get_tweet_by_id("1771236946049655268")
print(res.text)
print(res.media)

🇷🇺 | Footage (18+) of the moment the gunmen opened fire towards civilians at Crocus City Hall 
https://t.co/q9RxrvjQJt

[
    {
        'display_url': 'pic.twitter.com/q9RxrvjQJt',
        'expanded_url': 'https://twitter.com/MilitaryPOV/status/1771236946049655268/video/1',
        'id_str': '1771236844866293760',
        'indices': [94, 117],
        'media_key': '7_1771236844866293760',
        'media_url_https': 
'https://pbs.twimg.com/ext_tw_video_thumb/1771236844866293760/pu/img/Rv-i4Exm-eXQuuyj.jpg',
        'type': 'video',
        'url': 'https://t.co/q9RxrvjQJt',
        'additional_media_info': {'monetizable': False},
        'ext_media_availability': {'status': 'Available'},
        'sizes': {
            'large': {'h': 848, 'w': 480, 'resize': 'fit'},
            'medium': {'h': 848, 'w': 480, 'resize': 'fit'},
            'small': {'h': 680, 'w': 385, 'resize': 'fit'},
            'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}
        },
        'original_info': {'height': 848, 'width': 480, 'focus_rects': []},
        'video_info': {
            'aspect_ratio': [30, 53],
            'duration_millis': 29210,
            'variants': [
                {
                    'content_type': 'application/x-mpegURL',
                    'url': 
'https://video.twimg.com/ext_tw_video/1771236844866293760/pu/pl/sgGZmE-pDQXm0tnJ.m3u8?tag=12&container=cmaf'
                },
                {
                    'bitrate': 632000,
                    'content_type': 'video/mp4',
                    'url': 
'https://video.twimg.com/ext_tw_video/1771236844866293760/pu/vid/avc1/320x564/se3FJr-f361Xa7g7.mp4?tag=12'
                },
                {
                    'bitrate': 950000,
                    'content_type': 'video/mp4',
                    'url': 
'https://video.twimg.com/ext_tw_video/1771236844866293760/pu/vid/avc1/480x848/UhkMCeHh6uCcPGRT.mp4?tag=12'
                }
            ]
        }
    }
]

In [38]:
print(f"{res.text=}, {res.full_text=}")

res.text='I’m not certain as to whether @Tesla did a tug-of-war with the @cybertruck versus a snow cat during their
testing and homologation process so just in case they missed that test… We’ve got you covered 👍\n\nAlso if 
@elonmusk doesn’t repost this it will prove that he is, in fact, an… https://t.co/oi3o7FVSCy', res.full_text='I’m 
not certain as to whether @Tesla did a tug-of-war with the @cybertruck versus a snow cat during their testing and 
homologation process so just in case they missed that test… We’ve got you covered 👍\n\nAlso if @elonmusk doesn’t 
repost this it will prove that he is, in fact, an alien after all  \n\nEdit: the truck would have tugged much 
harder but we had a broken wheel sensor that kept throwing off the computer and in turn, limiting power. It was an 
easy fix'

In [40]:
print(res.media[0])

{
    'display_url': 'pic.twitter.com/oi3o7FVSCy',
    'expanded_url': 'https://twitter.com/heavydsparks801/status/1773207638698078280/video/1',
    'id_str': '1773205278521184256',
    'indices': [280, 303],
    'media_key': '13_1773205278521184256',
    'media_url_https': 'https://pbs.twimg.com/amplify_video_thumb/1773205278521184256/img/ekWrLyZW0K_obOHj.jpg',
    'type': 'video',
    'url': 'https://t.co/oi3o7FVSCy',
    'additional_media_info': {'monetizable': False},
    'ext_media_availability': {'status': 'Available'},
    'sizes': {
        'large': {'h': 886, 'w': 1542, 'resize': 'fit'},
        'medium': {'h': 689, 'w': 1200, 'resize': 'fit'},
        'small': {'h': 391, 'w': 680, 'resize': 'fit'},
        'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}
    },
    'original_info': {'height': 886, 'width': 1542, 'focus_rects': []},
    'video_info': {
        'aspect_ratio': [771, 443],
        'duration_millis': 36928,
        'variants': [
            {
                'content_type': 'application/x-mpegURL',
                'url': 
'https://video.twimg.com/amplify_video/1773205278521184256/pl/10r_hNdRO6sDbuyA.m3u8?tag=14&container=cmaf'
            },
            {
                'bitrate': 288000,
                'content_type': 'video/mp4',
                'url': 
'https://video.twimg.com/amplify_video/1773205278521184256/vid/avc1/468x270/-D56Ziq_qfbwbFX5.mp4?tag=14'
            },
            {
                'bitrate': 832000,
                'content_type': 'video/mp4',
                'url': 
'https://video.twimg.com/amplify_video/1773205278521184256/vid/avc1/626x360/1_W2YNhkEJCzvGQH.mp4?tag=14'
            },
            {
                'bitrate': 2176000,
                'content_type': 'video/mp4',
                'url': 
'https://video.twimg.com/amplify_video/1773205278521184256/vid/avc1/1252x720/rtVWb5gZbf3B4wJ-.mp4?tag=14'
            }
        ]
    }
}